In [1]:
import pandas as pd


In [3]:
pip install google

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [4]:
import pandas as pd

# Example: load a CSV file
data = pd.read_csv(r'C:\Users\LENOVO\Documents\B TECH SEM 5\ProjectSEC\clean_spam_email_dataset.csv')
print(data.head())


                                             message  label
0  Author: allison\nDate: Fri Jun 29 10:33:22 200...      0
1  I expect to be here. 713 276 7374. Maybe a win...      0
2  on mon escapenumber escapenumber escapenumber ...      0
3  ['free play whether better off for some of str...      1
4  Thanks for the reply. Please see embedded resp...      0


In [5]:
data_1 = data

In [6]:
data_1

,message,label
0,Author: allison\nDate: Fri Jun 29 10:33:22 200...,0
1,I expect to be here. 713 276 7374. Maybe a win...,0
2,on mon escapenumber escapenumber escapenumber ...,0
3,['free play whether better off for some of str...,1
4,Thanks for the reply. Please see embedded resp...,0
...,...,...
555220,It was so nice to hear from you. I am glad all...,0
555221,----- Richard B Sanders/HOU/ECT on 12/01/2000...,0
555222,Louise/John: These changes are consistent with...,0
555223,['Anatrim - the most effective flesh loss blen...,1


In [24]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nitishabharathi/email-spam-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'email-spam-dataset' dataset.
Path to dataset files: /kaggle/input/email-spam-dataset


In [25]:
dataset_path = r"/kaggle/input/email-spam-dataset"

In [26]:
import os

In [27]:
files = os.listdir(dataset_path)

In [28]:
for files in files :
    print(files)

lingSpam.csv
enronSpamSubset.csv
completeSpamAssassin.csv


In [29]:
df1 = pd.read_csv(os.path.join(dataset_path,"enronSpamSubset.csv"))

In [30]:
df1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Body', 'Label'], dtype='object')

In [34]:
df1 = df1[['Label', 'Body']]

In [35]:
df2 = pd.read_csv(os.path.join(dataset_path,"lingSpam.csv"))

In [41]:
df1

,Label,Body
0,1,Subject: stock promo mover : cwtd\n * * * urge...
1,1,Subject: are you listed in major search engine...
2,1,"Subject: important information thu , 30 jun 20..."
3,1,Subject: = ? utf - 8 ? q ? bask your life with...
4,1,"Subject: "" bidstogo "" is places to go , things..."
...,...,...
9995,0,"Subject: monday 22 nd oct\n louise ,\n do you ..."
9996,0,Subject: missing bloomberg deals\n stephanie -...
9997,0,Subject: eops salary survey questionnaire\n we...
9998,0,"Subject: q 3 comparison\n hi louise ,\n i have..."


In [37]:
df2.columns

Index(['Unnamed: 0', 'Body', 'Label'], dtype='object')

In [38]:
df2 = df2[['Label', 'Body']]

In [42]:
df2

,Label,Body
0,1,Subject: great part-time or summer job !\n \n ...
1,1,Subject: auto insurance rates too high ?\n \n ...
2,1,Subject: do want the best and economical hunti...
3,1,Subject: email 57 million people for $ 99\n \n...
4,1,Subject: do n't miss these !\n \n attention ! ...
...,...,...
2600,0,Subject: computationally - intensive methods i...
2601,0,Subject: books : a survey of american linguist...
2602,0,Subject: wecol ' 98 - - western conference on ...
2603,0,Subject: euralex ' 98 - revised programme\n \n...


In [36]:
df3 = pd.read_csv(os.path.join(dataset_path,"completeSpamAssassin.csv"))

In [39]:
df3.columns

Index(['Unnamed: 0', 'Body', 'Label'], dtype='object')

In [40]:
df3= df3[['Label','Body']]

In [43]:
df3

,Label,Body
0,1,\nSave up to 70% on Life Insurance.\nWhy Spend...
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...
2,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...
3,1,##############################################...
4,1,I thought you might like these:\n1) Slim Down ...
...,...,...
6041,0,empty
6042,0,___ ___ ...
6043,0,IN THIS ISSUE:01. Readers write\n02. Extension...
6044,0,empty


In [44]:
import pandas as pd

# Merge all three
data_3 = pd.concat([df1, df2, df3], ignore_index=True)

# (Optional) remove duplicates, if any


# Quick check
print(data_3.shape)
print(data_3['Label'].value_counts())


(18651, 2)
Label
0    11322
1     7329
Name: count, dtype: int64


In [47]:
data_3.duplicated().sum()

np.int64(0)

In [46]:
data_3 = data_3.drop_duplicates(subset=['Body']).reset_index(drop=True)

In [48]:
import re

def clean_for_cnn_v2(text):
    text = str(text).lower()

    # 0. Remove 'subject:' if it appears inside the message
    text = re.sub(r'\bsubject\s*:\s*', ' ', text)

    # 1. Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Replace URLs with a special token
    text = re.sub(r'http\S+|www\S+', ' url ', text)

    # 3. Replace email addresses with a token (but keep domain info)
    text = re.sub(r'[\w\.-]+@([\w\.-]+)', r' email_\1 ', text)

    # 4. Replace numbers with a generic token
    text = re.sub(r'\d+', ' num ', text)

    # 5. Keep useful punctuation ($, !, ?) but remove others
    text = re.sub(r'[^a-zA-Z0-9\s!?$]', ' ', text)

    # 6. Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply it
data_3['Body'] = data_3['Body'].apply(clean_for_cnn_v2)


In [49]:
data_3

,Label,Body
0,1,stock promo mover cwtd urgent investor trading...
1,1,are you listed in major search engines ? submi...
2,1,important information thu num jun num importan...
3,1,? utf num ? q ? bask your life with ? ? utf nu...
4,1,bidstogo is places to go things to do hello pr...
...,...,...
17566,0,num isilo tm num num for palm os pocket pc and...
17567,0,effector vol num no num november num num email...
17568,0,we have extended our free seat sale until thur...
17569,0,one ten num num num insignificant matters heav...


In [50]:
data_3 = data_3.rename(columns={'Body': 'message', 'Label': 'label'})


In [51]:
data_3

,label,message
0,1,stock promo mover cwtd urgent investor trading...
1,1,are you listed in major search engines ? submi...
2,1,important information thu num jun num importan...
3,1,? utf num ? q ? bask your life with ? ? utf nu...
4,1,bidstogo is places to go things to do hello pr...
...,...,...
17566,0,num isilo tm num num for palm os pocket pc and...
17567,0,effector vol num no num november num num email...
17568,0,we have extended our free seat sale until thur...
17569,0,one ten num num num insignificant matters heav...


In [9]:
import kagglehub

# Download latest version
path_kaggle = kagglehub.dataset_download("imdeepmind/preprocessed-trec-2007-public-corpus-dataset")

print("Path to dataset files:", path_kaggle)

Using Colab cache for faster access to the 'preprocessed-trec-2007-public-corpus-dataset' dataset.
Path to dataset files: /kaggle/input/preprocessed-trec-2007-public-corpus-dataset


In [10]:
trec = r"/kaggle/input/preprocessed-trec-2007-public-corpus-dataset"

In [12]:
import os

In [14]:
files = os.listdir(trec)

In [16]:
for files in files :
  print(files)

processed_data.csv


In [17]:
trec_data = pd.read_csv(os.path.join(trec,"processed_data.csv"))

In [18]:
trec_data

,label,subject,email_to,email_from,message
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che..."
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the..."
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...
...,...,...,...,...,...
75414,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,the reply for your request for a job place [le...,"""Gnitpick"" <gnitpick@flax9.uwaterloo.ca>","""Sydney Car Centre"" <Merrill8783@168city.com>",Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"Re: [R] Me again, about the horrible documenta...",Duncan Murdoch <murdoch@stats.uwo.ca>,Philippe Grosjean <phgrosjean@sciviews.org>,"For those who are interested, I just cook a li..."
75417,0,Re: [R] RODBC problem,<r-help@stat.math.ethz.ch>,=?iso-8859-1?Q?Bernhard_Wellh=F6fer?=\n\t<Bern...,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."


In [19]:
trec_data.columns

Index(['label', 'subject', 'email_to', 'email_from', 'message'], dtype='object')

In [20]:
trec_data = trec_data[['label', 'message']]


In [21]:
trec_data

,label,message
0,1,Content-Type: text/html;\nContent-Transfer-Enc...
1,0,"Hi, i've just updated from the gulus and I che..."
2,1,"Content-Type: text/plain;\n\tcharset=""iso-8859..."
3,1,"Hey Billy, \n\nit was really fun going out the..."
4,1,Content-Type: multipart/alternative;\n ...
...,...,...
75414,1,Content-Type: text/html;\nContent-Transfer-Enc...
75415,1,Content-Type: text/html;\nContent-Transfer-Enc...
75416,0,"For those who are interested, I just cook a li..."
75417,0,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan..."


In [22]:
import re
import pandas as pd

def clean_for_cnn(text):
    text = str(text).lower()

    # 1. Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Replace URLs with a special token
    text = re.sub(r'http\S+|www\S+', ' url ', text)

    # 3. Replace email addresses with a token (but preserve the domain separately)
    text = re.sub(r'[\w\.-]+@([\w\.-]+)', r' email_\1 ', text)

    # 4. Replace numbers with a generic token
    text = re.sub(r'\d+', ' num ', text)

    # 5. Keep punctuation that might be meaningful in spam (like $, !, ?)
    # Remove other special symbols
    text = re.sub(r'[^a-zA-Z0-9\s!?$]', ' ', text)

    # 6. Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply to your dataset
trec_data['message'] = trec_data['message'].apply(clean_for_cnn)


/tmp/ipython-input-407450841.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trec_data['message'] = trec_data['message'].apply(clean_for_cnn)


In [23]:
trec_data

,label,message
0,1,content type text html content transfer encodi...
1,0,hi i ve just updated from the gulus and i chec...
2,1,content type text plain charset iso num num co...
3,1,hey billy it was really fun going out the othe...
4,1,content type multipart alternative boundary ne...
...,...,...
75414,1,content type text html content transfer encodi...
75415,1,content type text html content transfer encodi...
75416,0,for those who are interested i just cook a lit...
75417,0,hello as i wrote i call sqlfetch channel t stu...


In [8]:
data_1.shape

(555225, 2)

In [9]:
data_3.shape

(17571, 2)

In [10]:
trec_data.shape

NameError: name 'trec_data' is not defined

In [55]:
data_2 = trec_data

In [11]:
data_2.shape

(75419, 2)

In [7]:
# Example paths (you can rename folders as you wish)
data_2.to_csv('/content/drive/MyDrive/data_2.csv', index=False)
data_3.to_csv('/content/drive/MyDrive/data_3.csv', index=False)


NameError: name 'data_2' is not defined

In [6]:
data_2 = pd.read_csv(r'C:\Users\LENOVO\Documents\B TECH SEM 5\ProjectSEC\data_2.csv')

In [7]:
data_3 = pd.read_csv(r'C:\Users\LENOVO\Documents\B TECH SEM 5\ProjectSEC\data_3.csv')

In [8]:

print("Shape of dataset 1 :",data_1.shape)
print("Shape of dataset 2 :",data_2.shape)
print("Shape of dataset 3 :",data_3.shape)

Shape of dataset 1 : (555225, 2)
Shape of dataset 2 : (75419, 2)
Shape of dataset 3 : (17571, 2)


In [9]:
import pandas as pd

# Example column name (change 'text' to your actual column)
col = 'message'

# 1️⃣ Combine all unique values from each dataset
set_1 = set(data_1[col])
set_2 = set(data_2[col])
set_3 = set(data_3[col])

# 2️⃣ Find intersections
common_12 = set_1.intersection(set_2)
common_13 = set_1.intersection(set_3)
common_23 = set_2.intersection(set_3)

# 3️⃣ Print results
print(f"Common between data_1 and data_2: {len(common_12)}")
print(f"Common between data_1 and data_3: {len(common_13)}")
print(f"Common between data_2 and data_3: {len(common_23)}")

# Optional: check if any value exists in all three
common_all = set_1.intersection(set_2, set_3)
print(f"Common in all three: {len(common_all)}")


Common between data_1 and data_2: 56
Common between data_1 and data_3: 4
Common between data_2 and data_3: 3
Common in all three: 1


In [10]:
# For example, view overlap between data_1 and data_2
overlap_df = data_1[data_1[col].isin(common_12)]
print(overlap_df.head())


                                                 message  label
4881   dictionaries are usually alphabetically ordere...      0
7085                                              lkjads      0
11283                                            lakjsdf      1
22272                                            edition      1
23776                                                but      1


In [11]:
col = 'message'  # or whichever column you checked earlier

# Remove overlaps
data_1_clean = data_1[~data_1[col].isin(set_2.union(set_3))]
data_2_clean = data_2[~data_2[col].isin(set_1.union(set_3))]
data_3_clean = data_3[~data_3[col].isin(set_1.union(set_2))]

# Check results
print(len(data_1_clean), len(data_2_clean), len(data_3_clean))


555119 73684 17563


In [12]:
for i, data in enumerate([data_1_clean, data_2_clean, data_3_clean], 1):
    print(f"Client {i} label distribution:\n", data['label'].value_counts(normalize=True), "\n")


Client 1 label distribution:
 label
0    0.596834
1    0.403166
Name: proportion, dtype: float64 

Client 2 label distribution:
 label
1    0.657918
0    0.342082
Name: proportion, dtype: float64 

Client 3 label distribution:
 label
0    0.626715
1    0.373285
Name: proportion, dtype: float64 



In [17]:
data_2_clean['label'].value_counts()

,count
label,
1,48478
0,25206


In [1]:
data_1

NameError: name 'data_1' is not defined

In [13]:
data_1 = data_1_clean
data_2 = data_2_clean
data_3 = data_3_clean

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, label_col='label'):
    # First, split off the train set
    df_train, df_temp = train_test_split(
        df,
        train_size=train_ratio,
        stratify=df[label_col],
        random_state=42
    )
    # Then, split the rest into validation and test sets
    val_test_ratio = val_ratio / (val_ratio + test_ratio)  # proportion for validation from temp
    df_val, df_test = train_test_split(
        df_temp,
        train_size=val_test_ratio,
        stratify=df_temp[label_col],
        random_state=42
    )
    return df_train, df_val, df_test

# # Load your datasets
# data_1 = pd.read_csv('data_1.csv')
# data_2 = pd.read_csv('data_2.csv')
# data_3 = pd.read_csv('data_3.csv')

# Split each dataset
train_1, val_1, test_1 = split_dataset(data_1, 0.7, 0.15, 0.15)
train_2, val_2, test_2 = split_dataset(data_2, 0.7, 0.15, 0.15)
train_3, val_3, test_3 = split_dataset(data_3, 0.6, 0.2, 0.2)  # For smallest dataset, use 60/20/20

# Optionally, save splits
# train_1.to_csv('data_1_train.csv', index=False)
# val_1.to_csv('data_1_val.csv', index=False)
# test_1.to_csv('data_1_test.csv', index=False)
# Repeat save operations for data_2 and data_3 as needed.


In [14]:
train_1.to_csv('data_1_train.csv', index=False)
val_1.to_csv('data_1_val.csv', index=False)
test_1.to_csv('data_1_test.csv', index=False)

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Example dataset structure (replace with your actual datasets)
# datasets = [data_1, data_2, data_3]

# Dummy example: Each dataset as list of dicts {'text': ..., 'label': ...}
# For actual datasets, load and preprocess accordingly

class SpamDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(
            item['text'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(item['label'])
        }

# Placeholder for each dataset
dataset1 = [ {'text': 'message1', 'label': 0}, ... ] # load your data
dataset2 = [ {'text': 'message2', 'label': 1}, ... ]
dataset3 = [ {'text': 'message3', 'label': 0}, ... ]

# Create dataset objects
train_dataset1 = SpamDataset(dataset1, tokenizer)
train_dataset2 = SpamDataset(dataset2, tokenizer)
train_dataset3 = SpamDataset(dataset3, tokenizer)

# Data loaders for federated clients
loader1 = DataLoader(train_dataset1, batch_size=16, shuffle=True)
loader2 = DataLoader(train_dataset2, batch_size=16, shuffle=True)
loader3 = DataLoader(train_dataset3, batch_size=16, shuffle=True)

# Load the model for each client
model1 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model2 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model3 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Now, in your federated framework, each dataset/model combination acts as a client
# For example, using Flower, you would define custom client classes and aggregate models

# --- (pseudo-code for FL) ---
# client1 = MyClient(model1, loader1)
# client2 = MyClient(model2, loader2)
# client3 = MyClient(model3, loader3)
# federated_learning([client1, client2, client3])



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-strea

In [25]:
pip install transformers

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import  get_scheduler
from sklearn.metrics import accuracy_score
import numpy as np

# Parameters
model_name = "distilbert-base-uncased"
max_length = 128
batch_size = 16
epochs = 3
lr = 5e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer & Model
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

# Custom Dataset
class SpamDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        message = self.data[idx]['message']
        label = self.data[idx]['label']
        encoding = self.tokenizer(message, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # remove batch dim
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Example usage, replace with your actual datasets
# datasets = [data_1, data_2, data_3]  each is list of dict {'text':..., 'label':...}
train_datasets = [dataset1, dataset2, dataset3]  # Load your data here

train_loaders = [DataLoader(SpamDataset(d, tokenizer, max_length), batch_size=batch_size, shuffle=True) for d in train_datasets]

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = epochs * sum(len(loader) for loader in train_loaders)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop for single client (Repeat separately per dataset in federated setting)
def train_one_epoch(dataloader):
    model.train()
    losses = []
    all_preds = []
    all_labels = []
    for batch in dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        losses.append(loss.item())
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    acc = accuracy_score(all_labels, all_preds)
    return np.mean(losses), acc

# Example federated training pseudocode snippet:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    for i, loader in enumerate(train_loaders):
        loss, acc = train_one_epoch(loader)
        print(f"  Client {i+1} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    # Add federated aggregation here (e.g., averaging weights between clients)

# After training, save or evaluate as needed


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1


TypeError: 'ellipsis' object is not subscriptable

In [17]:
data_1.shape

(555225, 2)

In [27]:
data_2.shape

(73684, 2)

In [28]:
data_3.shape

(17563, 2)

In [30]:
train_1

,message,label
50720,"\n""Oh, dear me, I assure you manage there is n...",1
339661,\n\nCNN Alerts: My Custom Alert\n\n\n\n\n\n\n ...,1
313405,hi i retrieve data from a mysql database conta...,0
402864,click here to claim click here to claim the ad...,1
393498,"Shirley, Please, register me for this conferen...",0
...,...,...
119481,Here is your second email. See ya tonight!! Lynn,0
324300,Mark - I'm out tomorrow to Dabhol partner meet...,0
60936,microsoft internet exploration sweepstakes bv ...,1
323244,Carol St. Clair EB 3889 713-853-3989 (Phone) 7...,0


In [22]:
import gc



In [23]:
gc.collect()

0

In [24]:
# ==============================
# STEP A: TOKENIZATION PIPELINE
# ==============================

import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ------------------------------
# 1️⃣ Combine all training sets to fit tokenizer
# ------------------------------
# Assuming you already loaded these DataFrames:
# data_1_train, data_2_train, data_3_train, etc.

all_train_texts = pd.concat([
    train_1['message'],
    train_2['message'],
    train_3['message']
], axis=0).astype(str).tolist()

# ------------------------------
# 2️⃣ Fit tokenizer
# ------------------------------
MAX_VOCAB_SIZE = 50000  # you can increase if data is very diverse
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
# Fit incrementally on each dataset
for df in [train_1, train_2, train_3]:
    tokenizer.fit_on_texts(df['message'].astype(str))

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save tokenizer for reuse
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

vocab_size = min(MAX_VOCAB_SIZE, len(tokenizer.word_index) + 1)
print(f"✅ Tokenizer fitted. Vocabulary size = {vocab_size:,}")

# ------------------------------
# 3️⃣ Determine optimal MAX_LEN (sequence length)
# ------------------------------
lengths = [len(x.split()) for x in all_train_texts]
# MAX_LEN = int(np.percentile(lengths, 95))  # 95th percentile length
MAX_LEN = 400
print(f"✅ Sequence max length set to: 400")

# ------------------------------
# 4️⃣ Helper function to tokenize & pad
# ------------------------------
def preprocess_data(df, tokenizer, max_len):
    X = tokenizer.texts_to_sequences(df['message'].astype(str).tolist())
    X = pad_sequences(X, 400, padding='post', truncating='post')
    y = df['label'].values
    return np.array(X), np.array(y)

# ------------------------------
# 5️⃣ Tokenize + pad each dataset (train, val, test) for all clients
# ------------------------------
clients = {
    "data_1": (train_1, val_1, test_1),
    "data_2": (train_2, val_2, test_2),
    "data_3": (train_3,val_3, test_3)
}

processed_data = {}

for client_name, (train_df, val_df, test_df) in clients.items():
    X_train, y_train = preprocess_data(train_df, tokenizer, MAX_LEN)
    X_val, y_val = preprocess_data(val_df, tokenizer, MAX_LEN)
    X_test, y_test = preprocess_data(test_df, tokenizer, MAX_LEN)

    processed_data[client_name] = {
        "X_train": X_train, "y_train": y_train,
        "X_val": X_val, "y_val": y_val,
        "X_test": X_test, "y_test": y_test
    }

    print(f"✅ {client_name}: {len(X_train)} train | {len(X_val)} val | {len(X_test)} test")

# ------------------------------
# 6️⃣ Save essential parameters for later use
# ------------------------------
np.savez_compressed("preprocessing_artifacts.npz",
                    vocab_size=vocab_size, max_len=MAX_LEN)

print("\n🎯 Step A completed successfully!")
print("Tokenizer, vocab size, and padded datasets are ready for model training.")


✅ Tokenizer fitted. Vocabulary size = 50,000
✅ Sequence max length set to: 400
✅ data_1: 388657 train | 83284 val | 83284 test
✅ data_2: 52793 train | 11313 val | 11313 test
✅ data_3: 10542 train | 3514 val | 3515 test

🎯 Step A completed successfully!
Tokenizer, vocab size, and padded datasets are ready for model training.


In [25]:
print(processed_data['data_1']['X_train'].shape)
print(processed_data['data_1']['y_train'][:10])


(388657, 400)
[1 0 1 1 1 0 0 0 1 0]


In [26]:
print(processed_data['data_2']['X_train'].shape)
print(processed_data['data_2']['y_train'][:10])

(52793, 400)
[1 0 1 1 1 1 1 1 1 1]


In [27]:
print(processed_data['data_3']['X_train'].shape)
print(processed_data['data_3']['y_train'][:10])

(10542, 400)
[0 0 0 0 1 0 0 1 1 0]


In [28]:
# ---------------------------
# STEP B: Baseline ML Pipelines
# ---------------------------
# Requirements:
# pip install scikit-learn xgboost matplotlib seaborn joblib

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, auc
)
from sklearn.pipeline import make_pipeline
import joblib
import xgboost as xgb

# ---------
# Assumptions:
# - You have data_1_train, data_1_val, data_1_test (DataFrames)
# - data_2_*, data_3_* likewise
# Each DF has columns: 'message' (cleaned text) and 'label' (0 or 1)
# ---------

clients = {
    "data_1": (train_1, val_1, test_1),
    "data_2": (train_2, val_2, test_2),
    "data_3": (train_3, val_3, test_3),
}

os.makedirs("baselines", exist_ok=True)

# ------------------------------
# 1) Fit a shared TF-IDF vectorizer on combined training texts
# ------------------------------
all_train_texts = pd.concat([df['message'] for (df,_,_) in clients.values()], axis=0).astype(str).tolist()

tfidf = TfidfVectorizer(
    max_features=50000,  # matches tokenizer cap; reduce if memory is tight
    ngram_range=(1,2),   # unigrams + bigrams
    min_df=5,            # ignore very rare tokens
    max_df=0.95
)
tfidf.fit(all_train_texts)
joblib.dump(tfidf, "baselines/tfidf_vectorizer.joblib")
print("Saved TF-IDF vectorizer -> baselines/tfidf_vectorizer.joblib")

# Helper to compute metrics
def eval_metrics(y_true, y_pred, y_prob=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc = roc_auc_score(y_true, y_prob) if y_prob is not None else None
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "roc_auc": roc}

# ------------------------------
# 2) Train per-client Logistic Regression & XGBoost
# ------------------------------
results = {}

for client_name, (train_df, val_df, test_df) in clients.items():
    print(f"\n=== Training baselines for {client_name} ===")
    X_train = tfidf.transform(train_df['message'].astype(str).tolist())
    y_train = train_df['label'].values
    X_val = tfidf.transform(val_df['message'].astype(str).tolist())
    y_val = val_df['label'].values
    X_test = tfidf.transform(test_df['message'].astype(str).tolist())
    y_test = test_df['label'].values

    client_result = {}

    # ---- Logistic Regression ----
    lr = LogisticRegression(
        C=1.0,
        solver='saga',        # good for large sparse data
        max_iter=200,
        class_weight='balanced',  # handles class imbalance
        n_jobs=-1,
        random_state=42
    )
    lr.fit(X_train, y_train)
    joblib.dump(lr, f"baselines/{client_name}_logreg.joblib")
    print("Saved", f"baselines/{client_name}_logreg.joblib")

    y_pred_lr = lr.predict(X_test)
    y_prob_lr = lr.predict_proba(X_test)[:, 1]
    metrics_lr = eval_metrics(y_test, y_pred_lr, y_prob_lr)
    client_result['logreg'] = metrics_lr
    print("LogReg test metrics:", metrics_lr)

    # ---- XGBoost ----
    # convert to DMatrix for xgboost
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss',
        n_jobs=-1,
        random_state=42
    )
    xgb_model.fit(
        X_train, y_train,
        early_stopping_rounds=20,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    joblib.dump(xgb_model, f"baselines/{client_name}_xgb.joblib")
    print("Saved", f"baselines/{client_name}_xgb.joblib")

    y_pred_xgb = xgb_model.predict(X_test)
    y_prob_xgb = xgb_model.predict_proba(X_test)[:,1]
    metrics_xgb = eval_metrics(y_test, y_pred_xgb, y_prob_xgb)
    client_result['xgb'] = metrics_xgb
    print("XGBoost test metrics:", metrics_xgb)

    results[client_name] = client_result

# ------------------------------
# 3) Aggregate & print results
# ------------------------------
print("\n\n===== SUMMARY RESULTS =====")
for client, res in results.items():
    print(f"\n-- {client} --")
    for model_name, metrics in res.items():
        print(f"{model_name:6s} | Acc: {metrics['accuracy']:.4f} F1: {metrics['f1']:.4f} ROC-AUC: {metrics['roc_auc']:.4f}")

# ------------------------------
# 4) Combined (global) evaluation: train on combined train sets and test on aggregated tests
# ------------------------------
print("\n=== Global baseline (train on combined clients) ===")
train_all = pd.concat([clients[c][0] for c in clients.keys()], axis=0)
val_all = pd.concat([clients[c][1] for c in clients.keys()], axis=0)
test_all = pd.concat([clients[c][2] for c in clients.keys()], axis=0)

X_train_all = tfidf.transform(train_all['message'].astype(str).tolist())
y_train_all = train_all['label'].values
X_val_all = tfidf.transform(val_all['message'].astype(str).tolist())
y_val_all = val_all['label'].values
X_test_all = tfidf.transform(test_all['message'].astype(str).tolist())
y_test_all = test_all['label'].values

global_lr = LogisticRegression(C=1.0, solver='saga', max_iter=200, class_weight='balanced', n_jobs=-1, random_state=42)
global_lr.fit(X_train_all, y_train_all)
joblib.dump(global_lr, "baselines/global_logreg.joblib")
y_pred = global_lr.predict(X_test_all)
y_prob = global_lr.predict_proba(X_test_all)[:,1]
print("Global LogReg:", eval_metrics(y_test_all, y_pred, y_prob))

global_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1,
    random_state=42
)
global_xgb.fit(X_train_all, y_train_all, early_stopping_rounds=20, eval_set=[(X_val_all, y_val_all)], verbose=False)
joblib.dump(global_xgb, "baselines/global_xgb.joblib")
y_pred_g = global_xgb.predict(X_test_all)
y_prob_g = global_xgb.predict_proba(X_test_all)[:,1]
print("Global XGBoost:", eval_metrics(y_test_all, y_pred_g, y_prob_g))

# ------------------------------
# 5) Plots: confusion matrix and ROC for the global best model (choose xgb here)
# ------------------------------
def plot_confusion(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.show()

def plot_roc(y_true, y_prob, title="ROC Curve"):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
    plt.plot([0,1],[0,1],'--', linewidth=0.8)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

# Use global_xgb as example
plot_confusion(y_test_all, y_pred_g, title="Global XGBoost - Confusion Matrix")
plot_roc(y_test_all, y_prob_g, title="Global XGBoost - ROC Curve")

print("\n✅ Baseline training & evaluation complete. Models saved under ./baselines")


MemoryError: 

In [30]:
# ---------------------------
# STEP B (Upgraded): Memory-Safe Baseline ML Pipelines
# ---------------------------
# Requirements:
# pip install scikit-learn xgboost matplotlib seaborn joblib

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, auc
)
from sklearn.pipeline import make_pipeline
import joblib
import xgboost as xgb

# ---------------------------
# Assumptions:
# - You already have train_1/val_1/test_1 etc.
# - Each DF has: 'message' (cleaned text) and 'label' (0 or 1)
# ---------------------------

clients = {
    "data_1": (train_1, val_1, test_1),
    "data_2": (train_2, val_2, test_2),
    "data_3": (train_3, val_3, test_3),
}

os.makedirs("baselines", exist_ok=True)

# ---------------------------
# 1) HashingVectorizer (constant memory)
# ---------------------------
vectorizer = HashingVectorizer(
    n_features=2**18,        # ~260k features, scales easily
    alternate_sign=False,    # keep positive feature values
    ngram_range=(1, 2),      # unigrams + bigrams
    norm='l2'
)
joblib.dump(vectorizer, "baselines/hashing_vectorizer.joblib")
print("✅ Using HashingVectorizer (constant memory). Saved to baselines/hashing_vectorizer.joblib")

# ---------------------------
# Helper for metrics
# ---------------------------
def eval_metrics(y_true, y_pred, y_prob=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc = roc_auc_score(y_true, y_prob) if y_prob is not None else None
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "roc_auc": roc}

def batch_transform(texts, batch_size=50000):
    from scipy.sparse import vstack
    batches = []
    for i in range(0, len(texts), batch_size):
        batch = vectorizer.transform(texts[i:i+batch_size])
        batches.append(batch)
    return vstack(batches)

# ---------------------------
# 2) Train per-client models
# ---------------------------
results = {}

for client_name, (train_df, val_df, test_df) in clients.items():
    print(f"\n=== Training baselines for {client_name} ===")

    # X_train = vectorizer.transform(train_df["message"].astype(str).tolist())
    # y_train = train_df["label"].values
    # X_val = vectorizer.transform(val_df["message"].astype(str).tolist())
    # y_val = val_df["label"].values
    # X_test = vectorizer.transform(test_df["message"].astype(str).tolist())
    # y_test = test_df["label"].values
    
    X_train = batch_transform(train_df["message"].astype(str).tolist())
    y_train = train_df["label"].values
    X_val = batch_transform(val_df["message"].astype(str).tolist())
    y_val = val_df["label"].values
    X_test = batch_transform(test_df["message"].astype(str).tolist())
    y_test = test_df["label"].values

    client_result = {}

    # ---- Logistic Regression ----
    lr = LogisticRegression(
        C=1.0,
        solver='saga',
        max_iter=200,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    )
    lr.fit(X_train, y_train)
    joblib.dump(lr, f"baselines/{client_name}_logreg.joblib")
    print(f"Saved baselines/{client_name}_logreg.joblib")

    y_pred_lr = lr.predict(X_test)
    y_prob_lr = lr.predict_proba(X_test)[:, 1]
    metrics_lr = eval_metrics(y_test, y_pred_lr, y_prob_lr)
    client_result["logreg"] = metrics_lr
    print("LogReg test metrics:", metrics_lr)

    # ---- XGBoost ----
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric="logloss",
        n_jobs=-1,
        random_state=42
    )
    xgb_model.fit(
        X_train, y_train,
        early_stopping_rounds=20,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    joblib.dump(xgb_model, f"baselines/{client_name}_xgb.joblib")
    print(f"Saved baselines/{client_name}_xgb.joblib")

    y_pred_xgb = xgb_model.predict(X_test)
    y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]
    metrics_xgb = eval_metrics(y_test, y_pred_xgb, y_prob_xgb)
    client_result["xgb"] = metrics_xgb
    print("XGBoost test metrics:", metrics_xgb)

    results[client_name] = client_result

# ---------------------------
# 3) Summary
# ---------------------------
print("\n\n===== SUMMARY RESULTS =====")
for client, res in results.items():
    print(f"\n-- {client} --")
    for model_name, metrics in res.items():
        print(f"{model_name:6s} | Acc: {metrics['accuracy']:.4f} F1: {metrics['f1']:.4f} ROC-AUC: {metrics['roc_auc']:.4f}")

# ---------------------------
# 4) Global (combined) model
# ---------------------------
print("\n=== Global baseline (train on combined clients) ===")
train_all = pd.concat([clients[c][0] for c in clients.keys()], axis=0)
val_all = pd.concat([clients[c][1] for c in clients.keys()], axis=0)
test_all = pd.concat([clients[c][2] for c in clients.keys()], axis=0)

X_train_all = vectorizer.transform(train_all["message"].astype(str).tolist())
y_train_all = train_all["label"].values
X_val_all = vectorizer.transform(val_all["message"].astype(str).tolist())
y_val_all = val_all["label"].values
X_test_all = vectorizer.transform(test_all["message"].astype(str).tolist())
y_test_all = test_all["label"].values

global_lr = LogisticRegression(C=1.0, solver="saga", max_iter=200, class_weight="balanced", n_jobs=-1, random_state=42)
global_lr.fit(X_train_all, y_train_all)
joblib.dump(global_lr, "baselines/global_logreg.joblib")
y_pred = global_lr.predict(X_test_all)
y_prob = global_lr.predict_proba(X_test_all)[:, 1]
print("Global LogReg:", eval_metrics(y_test_all, y_pred, y_prob))

global_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42
)
global_xgb.fit(X_train_all, y_train_all, early_stopping_rounds=20, eval_set=[(X_val_all, y_val_all)], verbose=False)
joblib.dump(global_xgb, "baselines/global_xgb.joblib")
y_pred_g = global_xgb.predict(X_test_all)
y_prob_g = global_xgb.predict_proba(X_test_all)[:, 1]
print("Global XGBoost:", eval_metrics(y_test_all, y_pred_g, y_prob_g))

# ---------------------------
# 5) Visualization
# ---------------------------
def plot_confusion(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.show()

def plot_roc(y_true, y_prob, title="ROC Curve"):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.4f}")
    plt.plot([0, 1], [0, 1], "--", linewidth=0.8)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

plot_confusion(y_test_all, y_pred_g, title="Global XGBoost - Confusion Matrix")
plot_roc(y_test_all, y_prob_g, title="Global XGBoost - ROC Curve")

print("\n✅ Baseline training & evaluation complete. Models saved under ./baselines")


✅ Using HashingVectorizer (constant memory). Saved to baselines/hashing_vectorizer.joblib

=== Training baselines for data_1 ===


MemoryError: Unable to allocate 256. MiB for an array with shape (33554432,) and data type float64